In [9]:
! pip install keras.models

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 301.8 kB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... error
  error: subprocess-exited-with-error
  
  × pip subprocess to install build dependencies did not run successfully.
  │ exit code: 1
  ╰─> [120 lines of output]
      Ignoring numpy: markers 'python_version >= "3.9"' don't match your environment
        Using cached Cython-0.29.37-py2.py3-none-any.whl.metadata (3.1 kB)
        Installing build dependencies: started
        Installing build dependencies: finished with status 'done'
        Getting requirements to build wheel: started
        Getting requirements to build wheel: finished with status 'done'
        Preparing metadata (pyproject.toml): started
        Preparing metadata (pyproject.toml): finished with status 'done'
        Using cached preshed-3.0.9-cp38-cp38-macosx_11_0_arm64.whl.metadata (2.2 kB)
        Installing build dependencies: started
        Installing build dependencies: finished wi

In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import GlobalAveragePooling2D, BatchNormalization, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import matplotlib.pyplot as plt

# Define paths
dataset_dir = "./Celebrity Faces Dataset/"  # Update with the path to your extracted dataset
train_dir = os.path.join(dataset_dir, "train")
test_dir = os.path.join(dataset_dir, "test")

# Preprocess dataset
image_size = (128, 128)
batch_size = 32

# Training data generator with augmentation
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=40,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    brightness_range=[0.8, 1.2],
    horizontal_flip=True,
    fill_mode="nearest",
    validation_split=0.2
)

# Training data generator
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode="categorical",
    subset="training"
)

# Validation data generator
validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode="categorical",
    subset="validation"
)

# Test data generator
test_datagen = ImageDataGenerator(rescale=1.0/255.0)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=False
)

# Get number of classes
num_classes = len(train_generator.class_indices)

# Create the model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
base_model.trainable = True

# Unfreeze the last few layers of the base model for fine-tuning
for layer in base_model.layers[:-10]:
    layer.trainable = False

# Build the model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    BatchNormalization(),
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Print model summary
model.summary()

# Class Weights
class_weights = compute_class_weight('balanced', classes=list(range(num_classes)), y=train_generator.classes)

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)
checkpoint = ModelCheckpoint("model_checkpoint_{epoch:02d}.h5", monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)

# Train the model
history = model.fit(
    train_generator,
    epochs=30,
    validation_data=validation_generator,
    verbose=1,
    class_weight={i: class_weights[i] for i in range(len(class_weights))},
    callbacks=[early_stopping, reduce_lr]
)

history = model.fit(
    train_generator,
    epochs=50,
    validation_data=validation_generator,
    verbose=1,
    class_weight={i: class_weights[i] for i in range(len(class_weights))},
    callbacks=[early_stopping, reduce_lr, checkpoint]
)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_generator, verbose=0)
print(f'\nTest accuracy: {test_accuracy * 100:.2f}%')

# Visualize training history
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train')
plt.plot(history.history['val_accuracy'], label='Validation')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train')
plt.plot(history.history['val_loss'], label='Validation')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.tight_layout()
plt.show()

# Load the pre-trained face detection model
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Load the trained CNN model
model = load_model('model_checkpoint_50.h5')  # Load the best model checkpoint

# Function to extract face embeddings from an image
def get_face_embeddings(image, model):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    embeddings = []
    for (x, y, w, h) in faces:
        face_roi = image[y:y+h, x:x+w]
        face_roi = cv2.resize(face_roi, (128, 128))
        face_roi = np.expand_dims(face_roi, axis=0) / 255.0

        # Extract embedding using the model
        embedding = model.predict(face_roi)[:, :-2]  # Exclude the final classification layers

        embeddings.append(embedding)

    return embeddings

# Load known face embeddings and labels (replace with your actual data)
known_embeddings = np.load('known_embeddings.npy')
known_labels = np.load('known_labels.npy')

# Function to recognize a face in a new image
def recognize_face(image, model):
    embeddings = get_face_embeddings(image, model)

    if len(embeddings) == 0:
        return "No face detected"

    distances = np.linalg.norm(known_embeddings - embeddings[0], axis=1)
    best_match_index = np.argmin(distances)
    predicted_label = known_labels[best_match_index]

    # Threshold for confidence
    threshold = 0.5 
    if distances[best_match_index] <= threshold:
        return f"Recognized as: {predicted_label}"
    else:
        return "Unknown person"



Found 1152 images belonging to 17 classes.
Found 288 images belonging to 17 classes.
Found 360 images belonging to 17 classes.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 global_average_pooling2d (  (None, 512)               0         
 GlobalAveragePooling2D)                                         
                                                                 
 batch_normalization (Batch  (None, 512)               2048      
 Normalization)                                                  
                                                                 
 dense (Dense)               (None, 512)               262656    
                                                                 
 batch_normalization_1 (Bat  (None, 512)               2048      
 chNormalization)                                                
                                                        

In [11]:
#test
# Example usage
image_path = 'path/to/your/image.jpg'
image = cv2.imread(image_path)

result = recognize_face(image, model)
print(result)

In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import GlobalAveragePooling2D, BatchNormalization, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import matplotlib.pyplot as plt

# Define paths
dataset_dir = "./Celebrity Faces Dataset/"  # Update with the path to your extracted dataset
train_dir = os.path.join(dataset_dir, "train")
test_dir = os.path.join(dataset_dir, "test")

# Preprocess dataset
image_size = (128, 128)
batch_size = 32

# Training data generator with augmentation
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=40,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    brightness_range=[0.8, 1.2],
    horizontal_flip=True,
    fill_mode="nearest",
    validation_split=0.2
)

# Training data generator
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode="categorical",
    subset="training"
)

# Validation data generator
validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode="categorical",
    subset="validation"
)

# Test data generator
test_datagen = ImageDataGenerator(rescale=1.0/255.0)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=False
)

# Get number of classes
num_classes = len(train_generator.class_indices)

# Create the model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
base_model.trainable = True

# Unfreeze the last few layers of the base model for fine-tuning
for layer in base_model.layers[:-10]:
    layer.trainable = False

# Build the model
model = models.Sequential([
    base_model,
    GlobalAveragePooling2D(),
    BatchNormalization(),
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Print model summary
model.summary()

# Class Weights
class_weights = compute_class_weight('balanced', classes=list(range(num_classes)), y=train_generator.classes)

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)
checkpoint = ModelCheckpoint("model_checkpoint_{epoch:02d}.h5", monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)

# Train the model
history = model.fit(
    train_generator,
    epochs=30,
    validation_data=validation_generator,
    verbose=1,
    class_weight={i: class_weights[i] for i in range(len(class_weights))},
    callbacks=[early_stopping, reduce_lr]
)

history = model.fit(
    train_generator,
    epochs=50,
    validation_data=validation_generator,
    verbose=1,
    class_weight={i: class_weights[i] for i in range(len(class_weights))},
    callbacks=[early_stopping, reduce_lr, checkpoint]
)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_generator, verbose=0)
print(f'\nTest accuracy: {test_accuracy * 100:.2f}%')

# Visualize training history
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train')
plt.plot(history.history['val_accuracy'], label='Validation')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train')
plt.plot(history.history['val_loss'], label='Validation')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.tight_layout()
plt.show()

# Load the pre-trained face detection model
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Load the trained CNN model
model = load_model('model_checkpoint_50.h5')  # Load the best model checkpoint

# Function to extract face embeddings from an image
def get_face_embeddings(image, model):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    embeddings = []
    for (x, y, w, h) in faces:
        face_roi = image[y:y+h, x:x+w]
        face_roi = cv2.resize(face_roi, (128, 128))
        face_roi = np.expand_dims(face_roi, axis=0) / 255.0

        # Extract embedding using the model
        embedding = model.predict(face_roi)[:, :-2]  # Exclude the final classification layers

        embeddings.append(embedding)

    return embeddings

# Load known face embeddings and labels (replace with your actual data)
known_embeddings = np.load('known_embeddings.npy')
known_labels = np.load('known_labels.npy')

# Function to recognize a face in a new image
def recognize_face(image, model):
    embeddings = get_face_embeddings(image, model)

    if len(embeddings) == 0:
        return "No face detected"

    distances = np.linalg.norm(known_embeddings - embeddings[0], axis=1)
    best_match_index = np.argmin(distances)
    predicted_label = known_labels[best_match_index]

    # Threshold for confidence
    threshold = 0.5 
    if distances[best_match_index] <= threshold:
        return f"Recognized as: {predicted_label}"
    else:
        return "Unknown person"



Found 1152 images belonging to 17 classes.
Found 288 images belonging to 17 classes.
Found 360 images belonging to 17 classes.


NameError: name 'models' is not defined